In [51]:
import cv2
import dlib
import sys
import numpy as np
%matplotlib inline

In [52]:
PREDICTOR_PATH = "/home/alvin/caffe/project/classify/shape_predictor_68_face_landmarks.dat"
CASCADE_PATH = "/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_alt.xml"
IMG_PATH = "/home/alvin/caffe/project/classify/test.png"

predictor = dlib.shape_predictor(PREDICTOR_PATH)
cascade = cv2.CascadeClassifier(CASCADE_PATH)

In [53]:
# detect face by cascade
# get landmarks by dlib
def get_landmarks(im):
    rects = cascade.detectMultiScale(im, 1.3, 5)
    x,y,w,h = rects[0]
    rect = dlib.rectangle(x, y, x+w, y+h)
    return np.matrix([[p.x, p.y] for p in predictor(im, rect).parts()])

In [54]:
# mark points ion image
def anotate_landmarks(im, landmarks):
    im = im.copy()
    for idx, point in enumerate(landmarks):
        # x = point[0, 0]
        # y = point[0, 1]
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos, fontFace = 
                    cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, fontScale = 0.4, color = (0, 255, 255))
        cv2.circle(im, pos, 5, color = (0,255, 255))
    return im

In [55]:
#im = cv2.imread(sys.argv[1],1)
im = cv2.imread(IMG_PATH,1)
cv2.namedWindow("Result", 0)
cv2.imshow("Result", im)
cv2.imshow("Result", anotate_landmarks(im, get_landmarks(im)))

In [56]:
## get 68 key points
landmarks = get_landmarks(im)
print("landmarks： ", landmarks.shape)
x_min = 10000
x_max = 0
y_min = 10000
y_max = 0

landmarks：  (68, 2)


In [57]:
for i in range(48, 67):
    x = landmarks[i, 0]
    y = landmarks[i, 1]
    if x <x_min:
        x_min = x
    if x > x_max:
        x_max = x
    if y < y_min:
        y_min = y
    if y > y_max:
        y_max = y

roi_width = x_max - x_min
roi_height = y_max - y_min

roi = im[y_min : y_max, x_min : x_max, 0 : 3]

if roi_width > roi_height:
    dst_len = 1.5 * roi_width
else:
    dst_len = 1.5 * roi_height

diff_xlen = dst_len - roi_width
diff_ylen = dst_len - roi_height

img_rows, img_cols,img_c = im.shape
x_dst = x_min
y_dst = y_min

if x_min - diff_xlen/2 >=0 and x_min + dst_len < img_cols:
    x_dst = x_min - diff_xlen/2
elif x_min - diff_xlen/2 < 0:
    x_dst = 0
else:
    x_dst = img_cols - dst_len #there is convension that img_cols > dst_len

if y_min - diff_ylen/2 >=0 and y_min + dst_len < img_rows:
    y_dst = y_min - diff_ylen/2
elif y_min - diff_ylen/2 < 0:
    y_dst = 0
else:
    y_dst = img_rows - dst_len #there is convension that img_rows > dst_len
    
y1 = int(y_dst)
y2 = int(y_dst + dst_len) 
x1 = int(x_dst)
x2 = int(x_dst + dst_len)

cv2.rectangle(im, (x_min, y_min), (x_max, y_max), (0,255,255), 4)
cv2.rectangle(im, (x1, y1), (x2, y2), (0,255,0), 4)
cv2.imshow("mark", im);
roi = im[y1 : y2, x1 : x2, 0 : 3]
cv2.imwrite('makr.png', im)
cv2.imwrite("roi.png", roi)
cv2.imshow("roi", roi)
cv2.waitKey(0)
cv2.destroyAllWindows()